In [ ]:
# Setting up CoinMarketCap API requests w/ my unique API Key
# If you're using my code from GitHub, make sure you change your data limit to allow max amounts -JDrummond 
    # from the command line:
    # jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10
# For the sake of this demo, I'm setting my API 'limit' to 5 just to keep my RAM unused

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '09bfc317-0595-49ac-8a8b-3c9766fa29e4',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
# let's import pandas with the alias of 'as'
# test .json_normalize to check if it's sorting correctly 
# it should be displaying in standard notation, we will change that later 

import pandas as pd

pd.json_normalize(data['data'])

In [ ]:
# if all is sorted correctly, set option display to max columns 
# if you're pulling in a lot of rows and not limited to 5, use:
    # pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)
pd.json_normalize(data['data'])

In [ ]:
# let's add a timestamp to show when pulls were fetched and place it in a 'df' variable 

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now', utc=True)
df

In [ ]:
# let's create a function to store all this so we can automate it later to a .csv
# If you don't want to create a .csv file, create a second dataframe so you can append to new table 
    # first, delete or comment out the if else function for the .csv append. then:
    #    df2 = pd.json_normalize(data['data'])
    #    df2['timestamp'] = pd.to_datetime('now', utc=True)
    #    df_append = pd.DataFrame(df2)
    #    df = pd.concat([df2, df_append])

def api_runner():
    global df
    from requests import Request, Session
    from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
    import json

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'5',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '09bfc317-0595-49ac-8a8b-3c9766fa29e4',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now', utc=True)
    df
    
    # write this pulled data into a .csv file 
    
    if not os.path.isfile(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv'):
        df.to_csv(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv', header = 'columns_names')
    else:
       df.to_csv(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv', mode = 'a', header=False) 

In [ ]:
# Setting our loop to 333 times because that is our daily limit with CMC 
# I'll set the sleep time for 10 seconds for the sake of this code, but if you really wanted this API, set it to 
    # whatever variable amount of time you want to use 333 times per day

import os #operating system
from time import time 
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner Pull Completed')
    sleep(10)
exit()

In [ ]:
# We can use pandas to read our .csv file here if we want
# save it into a new variable to call later if needed

df_csv = pd.read_csv(r'C:\Users\pc-47\Desktop\Python SQL\CMC_API.csv')
df_csv

In [ ]:
# let's clean the scientific notation so it's easier to read

pd. set_option('display.float_format', lambda x: '%.5f' % x)
df_csv

In [ ]:
# let's now group by 'name' of crypto by the average/mean so we can combine btc, eth, etc. 
# NOTE: you can't group by unless it's a mean, median, mode, etc

df_grouped = df_csv.groupby('name', sort=False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean()
df_grouped

In [ ]:
# We can't use pulled data to visualize unless we stack or pivot it to rows, not columns 
# So let's convert this data frame to a stack 
# NOTE: stacked data in pandas will be stored as a 'core.series' not in a data frame

df_stack = df_grouped.stack()
df_stack

In [ ]:
# Convert the series back to a data frame sorted by 'values' we create 

df_DataFrames = df_stack.to_frame(name='values')
df_DataFrames

In [ ]:
# Need to get rid of 'name' being used as the index, so let's reset the index to read our current state of our data 

index = pd.Index(range(30))

df_index = df_DataFrames.reset_index()
df_index

In [ ]:
# Let's rename the 'level_1' column and store it in a new variable 

df_rename = df_index.rename(columns={'level_1': 'percent_change'})
df_rename

In [ ]:
# let's change the 'percent_change' to only the hour to set up our x axis

df_rename['percent_change'] = df_rename['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'], ['1h', '24h', '7d', '30d', '60d', '90d'])
df_rename

In [ ]:
# let's start to visualize this data above with seaborn and matplotlib

import seaborn as sns
import matplotlib.pyplot as plt

sns.catplot(x='percent_change', y='values', hue='name', data=df_rename, kind='point')
plt.show()

In [ ]:
# let's just look at the price of our crypto from our original data frame .csv API data pulls  

df_price = df_csv[['name', 'quote.USD.price', 'timestamp']]
df_price

In [ ]:
# let's 'query' into this list now and just select 'Bitcoin' with the timestamps of when we pulled it

df_price = df_csv[['name', 'quote.USD.price', 'timestamp']]
df_price = df_price.query("name == 'Bitcoin'")
df_price

In [ ]:
# let's create a line plot to show the change of BTC over the time we pulled the data 
# ideally we should convert the timestamps to a better format 

sns.set_theme(style='darkgrid')

sns.lineplot(x='timestamp', y='quote.USD.price', data=df_price)
plt.show()